In [1]:
from pyspark.sql import SparkSession

# New API
spark = SparkSession\
        .builder\
        .master("local[1]")\
        .appName("team22")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",4)\
        .getOrCreate()
        
# Old API (RDD)
sc = spark.sparkContext

ModuleNotFoundError: No module named 'pyspark'

In [2]:
flight_data = spark\
    .read\
    .format("csv")\
    .option("header","true")\
    .load("/home/ubuntu/data/*.csv")

In [3]:
flight_data.printSchema()

root
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: string (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: string (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- CarrierDelay:

In [4]:
print(flight_data.count())

123534969


In [ ]:
flight_data = flight_data.drop("CancellationCode","Distance","AirTime","CRSElapsedTime","Diverted","SecurityDelay","LateAircraftDelay","UniqueCarrier","TaxiIn", "TaxiOut", "FlightNum", "TailNum","ActualElapsedTime","CarrierDelay" ,"WeatherDelay","NASDelay")

# Grandma Question

Filter Origin and Destination

In [ ]:
airports_Q1 = flight_data.filter(flight_data["Origin"]=="LA").filter(flight_data["Departure"]=="Honolulu")

Week day delay

In [ ]:
week_day_delay = airports_Q1.select("DayOfWeek", "ArrDelay", "DepDelay")
week_day_delay = week_day_delay.withColumn("Total delay", week_day_delay["ArrDelay"].cast("float")+week_day_delay["DepDelay"].cast("float"))
week_day_delay = week_day_delay.groupBy("DayOfWeek")

Monthly delay

In [ ]:
month_delay = airports_T1.select("DayofMonth", "ArrDelay", "DepDelay")
month_delay = month_delay.withColumn("Total delay", month_delay["ArrDelay"].cast("float")+month_delay["DepDelay"].cast("float"))
month_delay = month_delay.groupBy("DayofMonth")

# Least delays/flights

In [ ]:
airports_Q2 = flight_data.select("Origin", "DepDelay")